### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

%matplotlib inline

### Utilities

In [3]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000):
        with pd.option_context("display.max_columns", 1000):
            display(df)


def univariate_with_fr(df_raw, feature, df_sparedetails):
    '''
    -------input--------
    df_raw = The raw dataframe containing raw data - DataTill233064 without outliers
    feature = The feature from raw dataset for with failure rate is required, Allowed values-DATACENTER,
    FAILEDPARTITEMGROUP
    df_sparedetails = The dataset with spare part details - raw data is merged on this data to calulate installedbase,
    followed by the FR%.
    -------output-------
    pandas DataFrame with failed parts grouped by the feature containing Counts of FAILEDPARTMSF, percentage count of 
    FAILEDPARTMSF, InstalledBase and FR%
    '''
    # Create groupby obejct on the raw data with feature required:-
    df_raw_feature_group = df_raw.groupby(feature)
    # On the group object, aggregate on the counts of FAILEDPARTMSF, sort in descending
    df_failed_by_feature = df_raw_feature_group.count()[['FAILEDPARTMSF']].sort_values(by='FAILEDPARTMSF',ascending=False).reset_index()
    
    # Rename FAILEDPARTMSF to 'Count of FAILEDPARTMSF'
    df_failed_by_feature.rename(columns={'FAILEDPARTMSF':'Count of FAILEDPARTMSF'}, inplace=True)

    # Make percentage count of FAILEDMARTMSF for each feature as 'Count of FAILEDPARTMSF2'
    df_failed_by_feature['Count of FAILEDPARTMSF2']=(df_failed_by_feature['Count of FAILEDPARTMSF']/(df_failed_by_feature['Count of FAILEDPARTMSF'].sum()))*100

    # Prepare for merge
    left_on = feature
    if feature == 'DATACENTER':
        right_on = 'DATACENTERCODE'
    if feature == 'FAILEDPARTITEMGROUP':
        right_on = 'BOMTYPE'
    # Sum of spares count grouped by feature
    df_sparedetails_feature_group = df_sparedetails.groupby(right_on)
    df_sparedetails_by_feature = df_sparedetails_feature_group.sum()[['count']].sort_values(by='count',ascending=False).reset_index()
    
    # Merge with df_sparedetails on feature to calculate InstalledBase and FR%
    df_failed_by_feature = df_failed_by_feature.merge(df_sparedetails_by_feature, left_on=left_on, right_on=right_on)
    df_failed_by_feature.drop(labels=right_on, axis=1, inplace=True)

    # Rename count to 'InstalledBase'
    df_failed_by_feature = df_failed_by_feature.rename(columns={'count':'InstalledBase'})
    # Calculate 'FR%'
    df_failed_by_feature['FR%'] = df_failed_by_feature['Count of FAILEDPARTMSF']/df_failed_by_feature['InstalledBase']*100
    
    return df_failed_by_feature


def univariate_with_onhanddc(df_raw, feature):
    '''
    -------input--------
    df_raw = The raw dataframe containing raw data - DataTill233064 without outliers
    feature = The feature from raw dataset on which grouping is requried, Allowed values-FAILEDMANUFACTURER,
    SPAREMANUFACTURER, ISOUTOFSTOCK, ISOUTOFWARRANTY, COUNTRY, YearsGrp, FAILEDPARTMSF
    
    -------output-------
    pandas DataFrame with failed parts grouped by the feature containing Counts of FAILEDPARTMSF, percentage count of 
    FAILEDPARTMSF, Sum of ONHAND_DC
    '''
    # Create groupby obejct on the raw data with feature required:-
    df_raw_feature_group = df_raw.groupby(feature)
    # On the group object, aggregate on the counts of FAILEDPARTMSF, sort in descending
    df_failed_by_feature = df_raw_feature_group.count()[['FAILEDPARTMSF']].sort_values(by='FAILEDPARTMSF',ascending=False).reset_index()
    
    # Rename FAILEDPARTMSF to 'Count of FAILEDPARTMSF'
    df_failed_by_feature.rename(columns={'FAILEDPARTMSF':'Count of FAILEDPARTMSF'}, inplace=True)

    # Make percentage count of FAILEDMARTMSF for each feature as 'Count of FAILEDPARTMSF2'
    df_failed_by_feature['Count of FAILEDPARTMSF2']=(df_failed_by_feature['Count of FAILEDPARTMSF']/(df_failed_by_feature['Count of FAILEDPARTMSF'].sum()))*100

    # Find the Sum of ONHAND_DC for each Feature
    df_failed_by_feature['Sum of ONHAND_DC'] = df_raw_feature_group.sum()['ONHAND_DC'].loc[df_failed_by_feature[feature]].values
    
    return df_failed_by_feature



### Get the Data

In [ ]:
# Get the raw data - DataTill233064
print('LOADING DataTill233064 from Failure_Data_06292021_MoCaSPoV_v1.1....')
df_raw = pd.read_excel('Failure_Data_06292021_MoCaSPoV_v1.1.xlsx', sheet_name='DataTill233064', parse_dates=True)
# drop the outlier rows
df_raw = df_raw[df_raw['Outlier']==0]

# Get the Spare details data with age
print('LOADING Spare details data from SparesMSF_AgeDetails_07052021....')
df_SparesAgeDetails_raw = pd.read_excel('SparesMSF_AgeDetails_07052021.xlsx')